In [1]:
# This snippet of code properly adds the working source root path to python's path
# so you no longer have to install spykshrk through setuptools
import sys, os 
root_depth = 2
notebook_dir = globals()['_dh'][0]
root_path = os.path.abspath(os.path.join(notebook_dir, '../'*root_depth))
# Add to python's path
try:
    while True:
        sys.path.remove(root_path)
except ValueError:
    # no more root paths
    pass
sys.path.append(root_path)
# Alternatively set root path as current working directory
#os.chdir(root_path)

In [2]:
import numpy as np
import scipy as sp
import pandas as pd
import functools
import holoviews as hv
from holoviews import dim, opts
import enum

# Spykshrk modules for data analysis
from spykshrk.franklab.data_containers import FlatLinearPosition, SpikeFeatures, \
        EncodeSettings, pos_col_format, SpikeObservation
from spykshrk.franklab.generator.place.data_generator import UnitNormalGenerator, TetrodeUniformUnitNormalGenerator
from spykshrk.franklab.pp_decoder.util import normal_pdf_int_lookup, gaussian
from spykshrk.franklab.pp_decoder.pp_clusterless import OfflinePPEncoder, OfflinePPDecoder
from spykshrk.franklab.pp_decoder.visualization import DecodeVisualizer
from spykshrk.franklab.visualization import LinPosVisualizer, TetrodeVisualizer
from spykshrk.franklab.pp_decoder.util import apply_no_anim_boundary
from spykshrk.util import AttrDict

hv.extension('bokeh')
hv.extension('matplotlib')

hv.renderer('bokeh').theme = "dark_minimal"
import matplotlib.pyplot as plt
plt.style.use('dark_background')
plt.rcParams['figure.facecolor'] = 'black'

pd.set_option('display.precision', 4)
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 15)


In [6]:
# Encoding and decoding settings for both simulator and algorithm

@enum.unique
class Arm(enum.Enum):
    center = 1
    left = 2
    right = 3

@enum.unique
class Direction(enum.Enum):
    forward = 1
    reverse = 2
    
encode_settings = AttrDict({'sampling_rate': 1000,
                            'pos_bins': np.arange(0,800,1),
                            'pos_bin_edges': np.arange(0,800.1,1),
                            'pos_bin_delta': 1,
                            'pos_kernel': sp.stats.norm.pdf(np.arange(0,800,1), 400, 3),
                            'pos_kernel_std': 3, 
                            'mark_kernel_std': int(20), 
                            'pos_num_bins': 800,
                            'pos_col_names': [pos_col_format(ii, 800) for ii in range(800)],
                            'arm_coordinates': [[0, 100], [100, 250], [250,400], [400, 500], [500, 650], [650, 800]],
                            'wtrack_arm_coordinates': AttrDict(center=AttrDict(forward=AttrDict(x1=0, x2=100, len=100), 
                                                                               reverse=AttrDict(x1=400, x2=500, len=100)),
                                                               left=AttrDict(forward=AttrDict(x1=100, x2=250, len=150),
                                                                             reverse=AttrDict(x1=250, x2=400, len=150)),
                                                               right=AttrDict(forward=AttrDict(x1=500, x2=650, len=150),
                                                                              reverse=AttrDict(x1=650, x2=800, len=150))),
                            'vel': 3,
                            'spk_amp': 60})

decode_settings = AttrDict({'trans_smooth_std': 5,
                            'trans_uniform_gain': 0.001,
                            'time_bin_size': 10})
                            

In [7]:
def wtrack_simulate_arm_const_speed(arm_range, trial_time, trial_len, sampling_rate):
    pos_arm = np.arange(arm_range.x1, arm_range.x2, 
                        arm_range.len/sampling_rate/(trial_time*arm_range.len/(trial_len)))
    return pos_arm

def wtrack_simulate_trial_const_speed(arm1_range, arm1_enum, arm2_range, arm2_enum, trial_time, sampling_rate):
    pos_trial = np.array([])
    pos_trial_enum = np.array([])

    pos_arm = wtrack_simulate_arm_const_speed(arm1_range, trial_time, 
                                              arm1_range.len + arm2_range.len, sampling_rate)
    pos_trial = np.append(pos_trial, pos_arm)
    pos_trial_enum = np.append(pos_trial_enum, np.tile(arm1_enum, len(pos_arm)))

    pos_arm = wtrack_simulate_arm_const_speed(arm2_range, trial_time, 
                                              arm1_range.len + arm2_range.len, sampling_rate)
    pos_trial = np.append(pos_trial, pos_arm)
    pos_trial_enum = np.append(pos_trial_enum, np.tile(arm2_enum, len(pos_arm)))
    return pos_trial, pos_trial_enum

def wtrack_simulate_run_const_speed(arm1_dir, arm1_enum, arm2_dir, arm2_enum, trial_time, sampling_rate):
    pos_run = np.array([])
    pos_run_enum = np.array([])
    pos_run_dir_enum = np.array([])
    
    pos_trial, pos_trial_enum = wtrack_simulate_trial_const_speed(arm1_dir.forward, arm1_enum,
                                                                  arm2_dir.forward, arm2_enum, 
                                                                  trial_time, sampling_rate)
    pos_run = np.append(pos_run, pos_trial)
    pos_run_enum = np.append(pos_run_enum, pos_trial_enum)
    pos_run_dir_enum = np.append(pos_run_dir_enum, np.tile(Direction.forward, len(pos_trial)))

    pos_trial, pos_trial_enum = wtrack_simulate_trial_const_speed(arm2_dir.reverse, arm2_enum,
                                                                  arm1_dir.reverse, arm1_enum,
                                                                  trial_time, sampling_rate)
    pos_run = np.append(pos_run, pos_trial)
    pos_run_enum = np.append(pos_run_enum, pos_trial_enum)
    pos_run_dir_enum = np.append(pos_run_dir_enum, np.tile(Direction.reverse, len(pos_trial)))

    return pos_run, pos_run_enum, pos_run_dir_enum

def wtrack_simulate_series_const_speed(w_coor, trial_time, sampling_rate):
    pos_series = np.array([])
    pos_series_enum = np.array([])
    pos_series_dir_enum = np.array([])
    
    pos_run, pos_run_enum, pos_run_dir_enum = wtrack_simulate_run_const_speed(w_coor.center, Arm.center, 
                                                                              w_coor.left, Arm.left, 
                                                                              trial_time, sampling_rate)
    pos_series = np.append(pos_series, pos_run)
    pos_series_enum = np.append(pos_series_enum, pos_run_enum)
    pos_series_dir_enum = np.append(pos_series_dir_enum, pos_run_dir_enum)

    pos_run, pos_run_enum, pos_run_dir_enum = wtrack_simulate_run_const_speed(w_coor.center, Arm.center,
                                                                              w_coor.right, Arm.right,
                                                                              trial_time, sampling_rate)
    pos_series = np.append(pos_series, pos_run)
    pos_series_enum = np.append(pos_series_enum, pos_run_enum)
    pos_series_dir_enum = np.append(pos_series_dir_enum, pos_run_dir_enum)    
    
    return pos_series, pos_series_enum, pos_series_dir_enum
    

In [8]:
trial_time = 10

pos_epoch = np.array([])

pos_series, pos_series_enum, pos_series_dir_enum = wtrack_simulate_series_const_speed(encode_settings.wtrack_arm_coordinates, 
                                                                                      trial_time, encode_settings.sampling_rate)

pos_epoch = np.append(pos_epoch, pos_series)

pos_timestamp = np.arange(0, len(pos_epoch), 1)

pos_time = np.arange(0, pos_epoch.size/encode_settings.sampling_rate, 1/encode_settings.sampling_rate)

pos_vel = np.ones(len(pos_epoch)) * (pos_epoch[1] - pos_epoch[0]) / (pos_time[1] - pos_time[0])

# Duplicate
num_dup = 2
for ii in range(num_dup):
    pos_epoch = np.append(pos_epoch, pos_epoch)
    pos_timestamp = np.arange(0, len(pos_epoch), 1)
    pos_time = np.arange(0, pos_epoch.size/encode_settings.sampling_rate, 1/encode_settings.sampling_rate)
    pos_vel = np.append(pos_vel, pos_vel)
    pos_series_enum = np.append(pos_series_enum, pos_series_enum)
    pos_series_dir_enum = np.append(pos_series_dir_enum, pos_series_dir_enum)


In [9]:
linpos_flat = FlatLinearPosition.from_numpy_single_epoch(1, 1, pos_timestamp, pos_epoch, pos_vel, 
                                                         encode_settings.sampling_rate, 
                                                         encode_settings.wtrack_arm_coordinates)

linpos_flat['arm'] = pd.Series(index=linpos_flat.index, data=pos_series_enum, dtype='category')
linpos_flat['direction'] = pd.Series(index=linpos_flat.index, data=pos_series_dir_enum, dtype='category')

In [10]:
arm_colormap = dict(center='darkorange', left='pink', right='cyan')

direction_hatchmap = dict(forward='right_diagonal_line', reverse='left_diagonal_line')
mpl_direction_hatchmap = dict(forward='/', reverse='\\')

def wtrack_linear_plot_hook(plot, element, arm, direction):
    if hv.Store.current_backend == 'bokeh':
        plot.handles['glyph'].fill_color = arm_colormap[arm]
        plot.handles['glyph'].hatch_pattern = direction_hatchmap[direction]
        plot.handles['glyph'].line_color = None
        plot.handles['glyph'].hatch_color = 'grey'
    elif hv.Store.current_backend == 'matplotlib':
        element.opts(hatch=mpl_direction_hatchmap[direction], facecolor=arm_colormap[arm], clone=False)

def wtrack_linear_plot_init_hook(plot, element, arm, direction):
    if hv.Store.current_backend == 'bokeh':
        pass
    elif hv.Store.current_backend == 'matplotlib':
        plot_kwargs = plot.style.kwargs
        plot_kwargs['facecolor'] = color=arm_colormap[arm]
        plot.style = opts.Polygons(**plot_kwargs, hatch=mpl_direction_hatchmap[direction])

def pos_hook(plot, element):
    if hv.Store.current_backend == 'bokeh':
        pass
    elif hv.Store.current_backend == 'matplotlib':
        pass
    
def pos_init_hook(plot, element):
    if hv.Store.current_backend == 'bokeh':
        pass
    elif hv.Store.current_backend == 'matplotlib':
        pass
        
def wtrack_linear_plot_polygons(arm, direction, pos_time, w_coor):
    time_range = (pos_time[0], pos_time[-1])
    y_range = (w_coor[arm][direction].x1, w_coor[arm][direction].x2)
    time_total = time_range[1] - time_range[0]
    time_center = time_total/2 + time_range[0]
    y_total = y_range[1] - y_range[0]
    y_center = y_total/2 + y_range[0]
    
    box = hv.Box(time_center, y_center, (time_total, y_total))    
    
    init_hooks = [functools.partial(wtrack_linear_plot_init_hook, arm=arm, direction=direction)]
    
    hooks = [functools.partial(wtrack_linear_plot_hook, arm=arm, direction=direction)]
    
    if hv.Store.current_backend == 'bokeh':
        poly = hv.Polygons(box).opts(hooks=hooks)
    elif hv.Store.current_backend == 'matplotlib':
        poly = hv.Polygons(box).opts(initial_hooks=init_hooks)
    return poly

def plot_position(time, pos, color='royalblue', fig_size=400, frame_width=800, aspect=3):
    if hv.Store.current_backend == 'bokeh':
        return hv.Scatter((time, pos)).opts(color=color, frame_width=frame_width, aspect=aspect)
    elif hv.Store.current_backend == 'matplotlib':
        return hv.Scatter((time, pos)).opts(color=color, fig_size=fig_size, aspect=aspect)
        


In [11]:
hv.output(backend='matplotlib')
#hv.output(backend='matplotlib', fig='pdf')

w_coor = encode_settings.wtrack_arm_coordinates
wtrack_linear_plot_polygons(arm='center', direction='forward', pos_time=pos_time, w_coor=w_coor) * \
    wtrack_linear_plot_polygons(arm='left', direction='forward', pos_time=pos_time, w_coor=w_coor) * \
    wtrack_linear_plot_polygons( arm='left', direction='reverse', pos_time=pos_time, w_coor=w_coor) * \
    wtrack_linear_plot_polygons(arm='center', direction='reverse', pos_time=pos_time, w_coor=w_coor) * \
    wtrack_linear_plot_polygons(arm='right', direction='forward', pos_time=pos_time, w_coor=w_coor) * \
    wtrack_linear_plot_polygons(arm='right', direction='reverse', pos_time=pos_time, w_coor=w_coor) * \
    plot_position(pos_time, pos_epoch)
#hv.Scatter((pos_time, pos_epoch)).opts(color='royalblue', fig_size=400, aspect=2)

In [20]:
class WtrackLinposDecomposed:
    def __init__(self, wtrack_armcoord, bin_size=1):
        self.wtrack_armcoord = wtrack_armcoord
        self.bin_size = bin_size
        self.segment_order_cw = [('center', 'forward'),
                                 ('left', 'forward'), 
                                 ('left', 'reverse'),
                                 ('right', 'forward'),
                                 ('right', 'reverse'),
                                 ('center', 'reverse')]
        self.segment_order_ccw = [('center', 'forward'),
                                  ('right', 'forward'), 
                                  ('right', 'reverse'),
                                  ('left', 'forward'),
                                  ('left', 'reverse'),
                                  ('center', 'reverse')]
        self.armcoord_cw = self._segment_decomposed_with_buffer(self.segment_order_cw, wtrack_armcoord)
        self.armcoord_ccw = self._segment_decomposed_with_buffer(self.segment_order_ccw, wtrack_armcoord)
        self.armcoord_cw_num_bins = max([max((buffer.x1, buffer.x2)) for arm in self.armcoord_cw.values() 
                                         for direct in arm.values() for buffer in [direct.prev, direct.main, direct.next]])
        self.armcoord_ccw_num_bins = max([max((buffer.x1, buffer.x2)) for arm in self.armcoord_ccw.values() 
                                          for direct in arm.values() for buffer in [direct.prev, direct.main, direct.next]])
        self.wtrack_armcoord_cw = AttrDict({arm: AttrDict({direct: direct_v.main for direct, direct_v in arm_v.items()})
                                            for arm, arm_v in self.armcoord_cw.items()})
        self.wtrack_armcoord_ccw = AttrDict({arm: AttrDict({direct: direct_v.main for direct, direct_v in arm_v.items()})
                                             for arm, arm_v in self.armcoord_ccw.items()})
        self.simple_armcoord_cw, self.simple_armcoord_bins_cw = \
                self._create_wtrack_decomposed_simple_armcoord(self.armcoord_cw, bin_size)
        self.simple_armcoord_ccw, self.simple_armcoord_bins_ccw = \
                self._create_wtrack_decomposed_simple_armcoord(self.armcoord_ccw, bin_size)        
        
    @staticmethod
    def _segment_decomposed_with_buffer(segment_order, wtrack_arm_coord):
        seg_decomposed = AttrDict()
        seg_offset = 0
        for ii, seg in enumerate(segment_order):
            prev_seg = segment_order[ii-1]
            next_seg = segment_order[(ii+1)%len(segment_order)]

            prev_seg_len = wtrack_arm_coord[prev_seg[0]][prev_seg[1]].len
            main_seg_len = wtrack_arm_coord[seg[0]][seg[1]].len
            next_seg_len = wtrack_arm_coord[next_seg[0]][next_seg[1]].len

            prev_seg_start = seg_offset
            main_seg_start = prev_seg_start + prev_seg_len
            next_seg_start = main_seg_start + main_seg_len
            next_seg_end = next_seg_start + next_seg_len
            seg_total_len = prev_seg_len + main_seg_len + next_seg_len
            seg_offset += seg_total_len

            arm_dict = seg_decomposed.setdefault(seg[0], AttrDict())
            dir_dict = arm_dict.setdefault(seg[1], AttrDict(prev=AttrDict(x1=prev_seg_start, x2=main_seg_start, len=prev_seg_len), 
                                                            main=AttrDict(x1=main_seg_start, x2=next_seg_start, len=main_seg_len),
                                                            next=AttrDict(x1=next_seg_start, x2=next_seg_end, len=next_seg_len),
                                                            prev_seg=prev_seg,
                                                            next_seg=next_seg))
            
        return seg_decomposed
     
    @staticmethod
    def _create_wtrack_decomposed_simple_armcoord(decomp_armcoord, bin_size=1):
        simple_armcoord = [(direct.main.x1, direct.main.x2) for arm in decomp_armcoord.values() for direct in arm.values()]
        simple_armcoord.sort(key=lambda tup: tup[0])
        simple_armcoord_bins = np.array([])
        for seg in simple_armcoord:
            simple_armcoord_bins = np.append(simple_armcoord_bins, np.arange(seg[0], seg[1], bin_size))
        return simple_armcoord, simple_armcoord_bins


In [21]:
wtrack_decompose = WtrackLinposDecomposed(encode_settings.wtrack_arm_coordinates)

In [29]:
encode_settings_decomp = AttrDict(encode_settings)
encode_settings_decomp.pos_num_bins = wtrack_decompose.armcoord_cw_num_bins
encode_settings_decomp.pos_bins = np.arange(0, encode_settings_decomp.pos_num_bins)
encode_settings_decomp.pos_bin_edges = np.arange(0, encode_settings_decomp.pos_num_bins + 0.0001, 1)
encode_settings_decomp.pos_kernel_std = 3
encode_settings_decomp.pos_kernel = sp.stats.norm.pdf(np.arange(0, encode_settings_decomp.pos_num_bins, 1), 
                                                      encode_settings_decomp.pos_num_bins/2, 
                                                      encode_settings_decomp.pos_kernel_std)
encode_settings_decomp.pos_col_names = [pos_col_format(ii, encode_settings_decomp.pos_num_bins) 
                                        for ii in range(encode_settings_decomp.pos_num_bins)]
encode_settings_decomp.arm_coordinates = wtrack_decompose.simple_armcoord_bins_cw
encode_settings_decomp.wtrack_decomp_arm_coordinates = wtrack_decompose.wtrack_armcoord_cw


In [32]:
def wtrack_remap_to_decomposed(linpos_flat, wtrack_arm_coord, wtrack_decomposed_cw, wtrack_decomposed_ccw):
    linpos_arm_dir = linpos_flat.groupby(['arm', 'direction'])

    decomposed_linpos_cw = pd.DataFrame()
    decomposed_linpos_ccw = pd.DataFrame()
    
    decomposed_linpos = pd.DataFrame()
    
    for entry in linpos_arm_dir:
        key = entry[0]
        table = entry[1]
        arm_coord_range = wtrack_arm_coord[key[0].name][key[1].name]
        decomposed_range_cw = wtrack_decomposed_cw[key[0].name][key[1].name]
        decomposed_range_ccw = wtrack_decomposed_ccw[key[0].name][key[1].name]
        decomposed_table = table.copy()
        decomposed_table.loc[:,'linpos_cw'] = decomposed_table.loc[:, 'linpos_flat'] - arm_coord_range.x1 + decomposed_range_cw.main.x1
        decomposed_table.loc[:,'linpos_ccw'] = decomposed_table.loc[:, 'linpos_flat'] - arm_coord_range.x1 + decomposed_range_ccw.main.x1
        
        decomposed_linpos = decomposed_linpos.append(decomposed_table)
    
    decomposed_linpos.sort_index(inplace=True)
    return decomposed_linpos

In [33]:
decomposed_linpos = wtrack_remap_to_decomposed(linpos_flat, encode_settings.wtrack_arm_coordinates,
                                               wtrack_decompose.armcoord_cw,
                                               wtrack_decompose.armcoord_ccw)

In [34]:
decomposed_linpos.query('arm==@Arm.right and direction==@Direction.forward', engine='python')

In [35]:
all_decon_range = np.array([(order.x1, order.x2) for arm in wtrack_decompose.armcoord_cw.values()
                            for direct in arm.values() for order in [direct.prev, direct.main, direct.next]])
decon_min = all_decon_range.min()
decon_max = all_decon_range.max()

In [36]:
hv.output(backend='bokeh', size=150)

hv.Points(decomposed_linpos['linpos_cw'], kdims=[('samples', 'Samples'), 
                                                 ('linpos_cw', 'Position (cm)')]).opts(aspect=2, ylim=(decon_min, decon_max))

In [37]:
tet_gen = TetrodeUniformUnitNormalGenerator(sampling_rate=encode_settings.sampling_rate,
                                            num_marks=4,
                                            num_units=50,
                                            mark_mean_range=(60, 200),
                                            mark_cov_range=(20, 40),
                                            firing_rate_range=(5, 40),
                                            pos_field_range=wtrack_decompose.simple_armcoord_bins_cw,
                                            pos_field_bins=wtrack_decompose.simple_armcoord_bins_cw,
                                            pos_field_var_range=(2,10))

In [38]:
spk_amps, unit_spks = tet_gen.simulate_tetrode_over_pos(decomposed_linpos, col_name='linpos_cw')

In [39]:
spk_amps

In [40]:
tet_viz = TetrodeVisualizer(spk_amps, decomposed_linpos, unit_spks)
tet_viz.plot_color_3d_dynamic('linpos_cw', 'c00', 'c01')

In [41]:
decomposed_linpos.get_irregular_resampled(spk_amps)

In [51]:
%%time
#%%prun -r -s cumulative

# Setup encoding model and estimate the position distribution of each spike being encoded
encoder = OfflinePPEncoder(linflat=decomposed_linpos, enc_spk_amp=spk_amps, dec_spk_amp=spk_amps, 
                           #decode_settings=decode_settings,
                           #encode_settings=encode_settings, dask_worker_memory=min_worker_memory)
                           encode_settings=encode_settings, decode_settings=decode_settings,
                           linflat_col_name='linpos_cw'
                           )
observ_obj = encoder.run_encoder()

In [45]:
%%output backend='matplotlib' size=200 
%%opts Points (s=400 marker='x')

plt.style.use('dark_background')
plt.rcParams['figure.facecolor'] = 'black'

# Setup plot to visualize estimated position distribution

sel_distrib = observ_obj.loc[:, pos_col_format(0,encode_settings.pos_num_bins):         
                                 pos_col_format(encode_settings.pos_num_bins-1,
                                                encode_settings.pos_num_bins)]
    
sel_pos = observ_obj.loc[:, 'position']

max_prob = sel_distrib.max().max()
    
def plot_observ(ind):
        
    plot_list = []
    for ii in range(5):
        plot_list.append(hv.Curve(sel_distrib.iloc[ind+ii], extents=(0, 0, 2050, max_prob)))
        plot_list.append(hv.Points((sel_pos.iloc[ind+ii], [0.005])))
    return hv.Overlay(plot_list)
        
#Ind = Stream.define('stuff', ind=0)

dmap = hv.DynamicMap(plot_observ, kdims=['ind'])
dmap.redim.values(ind=list(range(0, len(observ_obj)-5, 5)))

In [ ]:
sel_distrib

In [ ]:
max